Price prediction

In [1]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, ParameterGrid, KFold
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm, trange 

C:\Users\arsen\AppData\Local\Temp\ipykernel_24544\93083308.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
df = pd.read_csv("apartment_corrected_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8978 entries, 0 to 8977
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Количество комнат  8978 non-null   int64  
 1   Площадь (м²)       8978 non-null   float64
 2   Этаж               8978 non-null   int64  
 3   Адрес              8978 non-null   object 
 4   Цена ($)           8978 non-null   int64  
 5   Цена (сом)         8978 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 421.0+ KB


In [3]:
features = ["Количество комнат", "Площадь (м²)", "Этаж"]
target = "Цена ($)"  # Или "Цена (сом)", если требуется

X = df[features]
y = df[target]

# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определяем сетку гиперпараметров для поиска
param_grid = {
    'iterations': [300],
    'learning_rate': [0.06],
    'depth': [9]
}
grid = list(ParameterGrid(param_grid))
kf = KFold(n_splits=3, shuffle=True, random_state=42)

best_score = float('inf')
best_params = None

print("Начало GridSearch:")

# Перебор всех комбинаций гиперпараметров с использованием tqdm для отображения прогресса
for params in tqdm(grid, desc="Параметрические комбинации", leave=True):
    cv_scores = []
    # 3-кратная кросс-валидация
    for train_index, val_index in kf.split(X_train):
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]
        # Создаем модель с текущими параметрами
        model = CatBoostRegressor(
            allow_writing_files=False,
            random_seed=42,
            task_type='GPU',          # Используем GPU для ускорения обучения
            eval_metric='RMSE',
            early_stopping_rounds=50,
            verbose=0,
            **params
        )
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val))
        y_pred_val = model.predict(X_val)
        score = mean_absolute_error(y_val, y_pred_val)
        cv_scores.append(score)
    mean_score = np.mean(cv_scores)
    if mean_score < best_score:
        best_score = mean_score
        best_params = params

print("Лучшие параметры:", best_params)
print("Лучший CV MAE:", best_score)

# Обучаем итоговую модель на полном обучающем наборе с найденными параметрами
best_model = CatBoostRegressor(
    allow_writing_files=False,
    random_seed=42,
    task_type='GPU',
    eval_metric='RMSE',
    early_stopping_rounds=50,
    verbose=100,  # Вывод прогресса обучения итоговой модели
    **best_params
)
best_model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Оценка модели на тестовой выборке
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Test MAE:", mae)
print("Test R2:", r2)

# Функция для предсказания цены квартиры по вводимым параметрам (без столбца "Адрес")
def predict(num_rooms, area, floor):
    input_data = pd.DataFrame({
        "Количество комнат": [num_rooms],
        "Площадь (м²)": [area],
        "Этаж": [floor]
    })
    return best_model.predict(input_data)[0]

# Пример использования функции
predicted_price = predict(3, 75, 5)
print("Предсказанная цена квартиры:", predicted_price)

Начало GridSearch:


Параметрические комбинации: 100%|██████████| 1/1 [00:03<00:00,  3.25s/it]


Лучшие параметры: {'depth': 9, 'iterations': 300, 'learning_rate': 0.06}
Лучший CV MAE: 20223.68676719971
0:	learn: 71603.9774764	test: 70875.7714375	best: 70875.7714375 (0)	total: 4.92ms	remaining: 1.47s
100:	learn: 29709.4239433	test: 32240.9077005	best: 32240.9077005 (100)	total: 427ms	remaining: 842ms
200:	learn: 27925.2181687	test: 31785.6239786	best: 31782.0259349 (195)	total: 836ms	remaining: 412ms
299:	learn: 26859.9028131	test: 31628.6915006	best: 31628.6084342 (298)	total: 1.25s	remaining: 0us
bestTest = 31628.60843
bestIteration = 298
Shrink model to first 299 iterations.
Test MAE: 19941.684934800858
Test R2: 0.8188256816445828
Предсказанная цена квартиры: 105852.07768671121


In [4]:
# Сохраняем модель в файл
best_model.save_model("apartment_catboost_model.cbm")